In [67]:
import re
from datasets import load_dataset

## Dataset extraction

In [ ]:
dataset = load_dataset("heegyu/namuwiki")

for item in dataset["train"]:
    if item["title"] == "마음의소리/등장인물":
        found_text = item["text"]
        break

## Util functions

In [71]:
def parse_content(text):
    heading_pattern = r"^(={2,})(.*?)\1$"
    parsed_content = {}
    current_heading = None

    for line in text.splitlines():
        match = re.match(heading_pattern, line)
        if match:
            heading_text = match.group(2).strip()
            current_heading = heading_text
            parsed_content[current_heading] = []
        elif current_heading is not None:
            parsed_content[current_heading].append(line.strip())
    return parsed_content


def clean_titles(parsed_data):
    parsed_data_dict = {}
    for heading, content in parsed_data.items():
        if content == []:
            continue
        cleaned_item = re.sub(r"~", "", heading)
        match = re.search(r"\|(.+?)\]\]", cleaned_item)
        if match:
            heading = match.group(1).replace("[", "").replace("]", "")
        else:
            heading = cleaned_item.replace("[", "").replace("]", "")
        parsed_data_dict[heading] = content
    return parsed_data_dict


def clean_contents(parsed_data):
    for key, contents in parsed_data.items():
        text = " ".join(contents)
        text = re.sub(r"\[''\], \[''\]", "", text)  # Remove empty lists
        text = re.sub(r"\[\[파일:.*?\.(?:jpg|png)\]\]", "", text)         
        text = text.replace("''", "").replace("'", "").replace("[", "").replace("]", "")
        text = re.sub(r"파일:.*?\|.*?=.*? ", "", text)  # Remove specific file links
        url_pattern = r"http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
        text = re.sub(url_pattern, "", text)  # Remove general URLs
        text = re.sub(r"\|(.*)", r"\1", text)  # Extract text after "|"
        text = text.strip()  # Remove leading and trailing whitespace
        text = re.sub(r"\[\[(.*?)\]\]", r"\1", text)  # Extract text inside "[[]]"
        text = text.replace("[*", "")  # Remove "[*]"
        parsed_data[key] = text 

    return parsed_data


## Dataset cleaning

In [75]:
parsed_data = parse_content(found_text)
parsed_data = clean_titles(parsed_data)
parsed_data = clean_contents(parsed_data)
parsed_data

{'개요': '마음의소리의 등장인물을 정리한 문서.',
 '조석의 가족': '조석의 가족 및 친척들. 현재 시점에서는 조석, 형 조준, 부모님, 아내 애봉이, 딸인 율봉이와 휘봉이, 장인장모, 처남 최고봉, 그리고 친척 동생인 쌍둥이들이 자주 등장한다.  사실 이들 외에 본편 31화에 여동생이 한 번 등장했다. 그런데 조석 본인이 1권 단행본 부록 미공개 콘티 만화에서 어머니에게 "(소재로 쓰게) 동생 하나만 낳아주시지 그러셨어요."라고 언급한 걸 보면 조석에게 친동생은 없는 듯. 마음의소리가 초창기에는 실화를 기반으로 한 에피소드도 많았지만 허구의 에피소드도 많았던 점을 감안하면 여동생은 가상의 인물이거나 친척 동생을 모티브로 한 듯하다.  참고로 작중에서 요리를 잘하는 큰어머니* 부대찌개하라며 명절 음식을 매우 잘하는 것으로 묘사되었다. 등 친척들을 제외하면 요리를 제대로 할 줄 아는 사람이 조철왕 밖에 없다.* 조석은 연재 초기에 종종 자취요리를 망치는 묘사가 있었고, 엄마와 애봉이는 요리치 속성이 있으며, 조준은 밥을 지어와도 죽으로 오해받거나 치킨집을 창업해놓고 자기가 만든 치킨맛이 똥맛이랏니 다른데서 시켜먹을 정도다. 그나마 치킨집으로 어느정도 먹고 살던 조철왕은 어느정도 준수한듯.',
 '조석': '작가 본인을 투영한 페르소나 캐릭터이자 본 작품의 주인공. 네이버 앱피소드 광고의 성우는 홍범기, 마음의소리(TVA)애니메이션과 마음의소리(게임)|모바일 게임에서의 성우는 정재헌, 마음의 소리(시트콤)|시트콤판 배우는 이광수(배우)|이광수, 마음의 소리 리부트: 얼간이들|리부트판 배우는 성훈(배우)|성훈. 800화 특집 중 등장하는 영화판 배역은 덴젤 워싱턴(...).  전에는 JS라는 필명을 사용했는데 정식 연재가 시작되면서 담당자가 작가 실명을 그대로 올린 바람에 그냥 본인 이름 그대로 사용 중이다. 그런데 닮았다고 하면 싫어한다. 이는 작가가 조석이란 캐릭터를 일부러 못생기게 그렸기 때문이다. 웹툰에서 공개된 내용인데 팬들 앞에서 자신의 모습이 아